# Arbeit mit Selenium

Die Arbeit mit Selenium erfordert etwas Übung. Aber der Zeitaufwand lohnt sich. Es gibt mit Selenium kaum ein Webdienst der nicht scrapbar wird. Beginnen wir aber wie üblich mit der Dokumentation. Sie ist im Falle von Selenium sehr hilfreich. Ihr findet [sie hier](http://selenium-python.readthedocs.io/). Und [hier](http://selenium-python.readthedocs.io/locating-elements.html). 

Um Selenium kennenzulernen, gehen wir zurück zu unserem Beispiel der Lehren: https://www.berufsberatung.ch/dyn/show/2930. Nun wollen wir keine URLs generieren, um unsere Inhalte zu finden. Wir wollen stattdessen mit der Site interagieren. So sollten wir alle Einträge bekommen. BeautifulSoup werden wir trotzdem noch dazu nehmen. Denn Selenium liest keine Inhalte aus. Die Library lässt uns einfach mit dem Webdienst interagieren.

Beginnen wir mit den Imports

In [107]:
from bs4 import BeautifulSoup
import requests
import time  #bremst die Suchabfrage
import datetime #gibt die Möglichkeit Daten effizienter zu manipulieren
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

Und dann schicken wir Selenium auf die Seite.

In [108]:
#Wir starten den Browser:
driver = webdriver.Chrome('/usr/local/bin/chromedriver')
#Wir besuchen die Site
driver.get("https://www.berufsberatung.ch/dyn/show/2930")

## Elemente ansteuern
Nun suchen wir mit dem Inspector die Elemente, die wir ansteuern wollen.

In [109]:
# HTML-Code der Webseite untersuchen. Was löst das Dropdown-Menu aus?
# Aantwort: class = "fs-autocomplete-trigger".....

In [110]:
driver.find_element_by_class_name("fs-autocomplete-trigger").click()

In [111]:
# Test mit einer Berufs-ID
# HTML-Code checken für Abdecker > sw_453
driver.find_element_by_id("sw_453").click()

In [112]:
# Auf den Butten "TREFFER ANZEIGEN" klicken
driver.find_element_by_id("uxfs-action").click()

In [113]:
test = driver.page_source
# test       # Zeigt den HTML Code an
type(test) # Es handelt sich um einen String.

str

In [114]:
# Wie kann der HTML-Code, respektive String, abgespeichert werden?
file = open("berufsberatung_abdichter.html", "w") # leeres file öffnen und benennen, als txt oder html
file.write(test)                                   # leeres File füllen mit dem Strin "test"
file.close()

## Liste mit den ID-Nummern generieren
Aber wir wollen alle Ergänzen. Holen wir deshalb die Nummern nochmals mit BeautifulSoup und kreieren die Liste ids = [....]

In [115]:
r = requests.get('https://www.berufsberatung.ch/dyn/show/2930')
soup = BeautifulSoup(r.text, 'html.parser')

In [116]:
# Test
# soup.find('ul',{'class':'ui-autocomplete full-list '})

In [117]:
meine_id_list = soup.find('ul',{'class':'ui-autocomplete full-list '}).find_all('a')

In [118]:
meine_id_list[0]['data-id']

'453'

In [119]:
ids=[]
for tag in meine_id_list:
    tag = "sw_" + tag['data-id']
    ids.append(tag)

In [120]:
# Test. Die ersten fünf Elemente anzeigen
ids[:5]

['sw_453', 'sw_452', 'sw_55', 'sw_7617', 'sw_7618']

Testen wir es mit den ersten fünf Einträgen

In [121]:
for elem in ids[:5]:
    print(elem)
    time.sleep(.5) #damit es nicht zu schnell geht
    driver.find_element_by_class_name("fs-autocomplete-trigger").click()
    time.sleep(.5)
    driver.find_element_by_id(elem).click()  

sw_453
sw_452
sw_55
sw_7617
sw_7618


In [122]:
# vorher haben wir nur die Lehrstellen geadded, jetzt lösen wir die Suche aus
driver.find_element_by_id("uxfs-action").click()  

Zeigen wird alle Ergebnisse an

In [123]:
# es werden nicht alle Suchresultate angezeigt. Man muss noch auf "Weitere anzeigen" klicken
driver.find_element_by_id("aSearchPaging").click()

Speichern wir die Ergebnisse ab

In [96]:
text = driver.page_source

In [97]:
def lehrstellen(html):
    
    soup = BeautifulSoup(html, 'html.parser')
    
    ortsliste = soup.find('div', {'class':'resultpart result-body'})\
                    .find_all('div', {'class':'display-table-cell table-col-3'})
    
    firmenliste = soup.find('div', {'class':'resultpart result-body'})\
                    .find_all('div', {'class':'display-table-cell bold company data-id table-col-1'})
    
    jahresliste = soup.find('div', {'class':'resultpart result-body'})\
                    .find_all('div', {'class':'display-table-cell float-left-for-sd table-col-4'})
        
    anzahlliste = soup.find('div', {'class':'resultpart result-body'})\
                    .find_all('div', {'class':'display-table-cell text-align-center float-left-for-sd table-col-5'})
    
    lst = []
    
    for ort, firma, jahr, anzahl in zip(ortsliste,firmenliste,jahresliste, anzahlliste):
        
        mini_dict = {'Ort':ort.text,
                     'Firma':firma.text,
                     'Jahr':jahr.text,
                     'Anzahl':int(anzahl.text.replace(' Lehrstelle(n)\n','').replace('\n',''))}
        lst.append(mini_dict)
    
    return lst

In [ ]:
# !!! Backslash \ ermöglicht einen Zeilenumbruch

In [98]:
lehrstellen(text)

[{'Ort': 'Aadorf (TG)',
  'Firma': 'Jakob Tanner AG',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Aesch BL (BL)',
  'Firma': 'peressini roofing AG',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Alpnach Dorf (OW)',
  'Firma': 'Abdichtungsbau Durrer AG',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Altendorf (SZ)',
  'Firma': 'Marty Stefan AG',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Altstätten SG (SG)',
  'Firma': 'Arthur Müggler & Co. AG',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Altstätten SG (SG)',
  'Firma': 'Räss AG',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Amriswil (TG)',
  'Firma': 'Vogel Dach- und Fassadenbau AG',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Amriswil (TG)',
  'Firma': 'Weber Bedachungen',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Bad Ragaz (SG)',
  'Firma': 'Bürer Flachdach AG',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Bassersdorf (ZH)',
  'Firma': 'Carl Meier Sohn AG',
  'Jahr': '2019',
  'Anzahl': 1},
 {'Ort': 'Bätterkinden (BE)',
  'Firma': 'E. J

In [99]:
pd.DataFrame(lehrstellen(text))

,Anzahl,Firma,Jahr,Ort
0,1,Jakob Tanner AG,2019,Aadorf (TG)
1,1,peressini roofing AG,2019,Aesch BL (BL)
2,1,Abdichtungsbau Durrer AG,2019,Alpnach Dorf (OW)
3,1,Marty Stefan AG,2019,Altendorf (SZ)
4,1,Arthur Müggler & Co. AG,2019,Altstätten SG (SG)
5,1,Räss AG,2019,Altstätten SG (SG)
6,1,Vogel Dach- und Fassadenbau AG,2019,Amriswil (TG)
7,1,Weber Bedachungen,2019,Amriswil (TG)
8,1,Bürer Flachdach AG,2019,Bad Ragaz (SG)
9,1,Carl Meier Sohn AG,2019,Bassersdorf (ZH)


## Bringen wir alles zusammen

ACHTUNG nicht zu schnell weiterklicken: (*) muss verschwinden.  
statt lxml muss html.parser stehen

In [ ]:
#Funktion, um nur die Informationen herauszuziehen, die uns interessieren
def lehrstellen(html):
    
    soup = BeautifulSoup(html, 'html.parser')
    try:
        ortsliste = soup.find('div', {'class':'resultpart result-body'})\
                    .find_all('div', {'class':'display-table-cell table-col-3'})
    
        firmenliste = soup.find('div', {'class':'resultpart result-body'})\
                    .find_all('div', {'class':'display-table-cell bold company data-id table-col-1'})
    
        jahresliste = soup.find('div', {'class':'resultpart result-body'})\
                    .find_all('div', {'class':'display-table-cell float-left-for-sd table-col-4'})
        
        anzahlliste = soup.find('div', {'class':'resultpart result-body'})\
                    .find_all('div', {'class':'display-table-cell text-align-center float-left-for-sd table-col-5'})
    
        lehrstelle = soup.find('ul',{'class':'ui-autocomplete full-list '})\
                    .find_all('a')
        lst = []
    
        for ort, firma, jahr, anzahl,lehr in zip(ortsliste,firmenliste,jahresliste, anzahlliste,lehrstelle):
        
            mini_dict = {'Ort':ort.text,
                     'Firma':firma.text,
                     'Jahr':jahr.text,
                     'Anzahl':int(anzahl.text.replace(' Lehrstelle(n)\n','').replace('\n','')),
                     'Lehrstelle':lehr['data-value']}
            lst.append(mini_dict)
    
        return pd.DataFrame(lst).to_csv("d/"+str(datetime.datetime.now())+".csv")
    
    except:
        return pd.DataFrame([{'Ort':'Keine Treffer',
                'Firma':'Keine Treffer',
                'Jahr':'Keine Treffer',
                'Anzahl':'Keine Treffer'}])
    
#Bauen wir Listen aller Job-IDs
r = requests.get('https://www.berufsberatung.ch/dyn/show/2930')
soup = BeautifulSoup(r.text, 'html.parser')
ids = []
for elem in soup.find('ul',{'class':'ui-autocomplete full-list '}).find_all('a'):
    elem = "sw_" + elem['data-id']
    ids.append(elem)
    
#Teilen wir diese Listen mit Länge von je 5 Teilen. 
#Das habe ich nicht selber geschrieben, sondern hier geholt:
#https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
idslst = [ids[i:i + 5] for i in range(0, len(ids), 5)]

count = 0
for ids in idslst:

    #Starten wir den Chrome-Browser und besuchen die Site
    driver = webdriver.Chrome('/usr/local/bin/chromedriver')
    driver.get("https://www.berufsberatung.ch/dyn/show/2930")

    #Bereiten wir die Suche vor    
    for elem in ids:
        time.sleep(1) #damit es nicht zu schnell geht
        driver.find_element_by_class_name("fs-autocomplete-trigger").click()
        time.sleep(1)
        driver.find_element_by_id(elem).click()
    
    #Suchen wir
    time.sleep(1)
    driver.find_element_by_id("uxfs-action").click()

    #Nun nun sorgen wir dafür, dass alle Ergebnisse anzeigt werden.
    exists = 1
    while(exists==1):
        loadmore = driver.find_element_by_id("aSearchPaging")
        if loadmore.text == "MEHR ERGEBNISSE ANZEIGEN":
            driver.find_element_by_id("aSearchPaging").click()
            time.sleep(1)
        else:
            exists = 0
            
    print(count)
    count += 1
    
    lehrstellen(driver.page_source)
    driver.close()

0
1
2
3


Kreieren wir ein kleine .py File und lösen es von der Commandline aus.